Summary: Experiments with heirachichal/multi-level modelling to account for in-group variance of repeated measures from schools. Several versions of these analyses/experiments were produced iteratively, the final of which contributed to a written report and can be found in `plots_for_multi_level_write_up.ipynb` 

Notes:
* All models incorrectly assume independence of within-school observations and thus variance estimates/confidence intervals are not accurate. The observations are weekly rates, thus are auto-correlated. This relationship is explored in the notebook `autocorr_tests.ipynb`.
* Bayesian sampling is used to estimate the model parameters (this method is later abandoned as being unnecessarily complex). 
* Multiple specifications of the logistic regression model are tested and the lowest BIC selected. This is somewhat arbitrary model selection, and several of the model variants are obviously functionally flawed i.e. increased CO2 decreasing illnesses or greater covid rates in the population decreasing illnesses etc.
* LOESS curves are again used as a comparator. The fit is arbitrary - it's based on the default settings of ggplot, and are not based on an interpretable or statistically sound model, thus shouldn't be considered reliable when interpreting the effects of either ACTs (or absence of ACTs). 
* None of the analyses explore the high-leverage outlying observations

In [ ]:
library(tidyverse)
library(tidybayes)
library(bigrquery)
library(brms)
library(ggmcmc)
library(loo)
library(lubridate)
library(gridExtra)
library(grid)
library(bayesplot)
library(marginaleffects)
library(table1)

hepa_school_codes <- c("H01", "H02", "H03", "H04", "H05", "H06", "H07", "H08", 
                       "H09", "H10", "H11")
control_school_codes <- c("C01", "C02", "C03", "C04", "C05", "C09", "C10", 
                          "C11", "C12", "C13", "C14")
study_schools <- c(hepa_school_codes, control_school_codes)

project_id="yhcr-prd-phm-bia-core" 
attendance_sql <- "SELECT * FROM `yhcr-prd-phm-bia-core.CY_CLASS_ACT.attendance`"
attendance_table <- bq_project_query(project_id, attendance_sql)

attendance <- bq_table_download(attendance_table) %>%
    filter(School_AnonID %in% study_schools) %>% 
    filter(WeekStart <= as.Date("2022-04-01")) %>%
    filter(pct_in_school > 0) 

attendance_data_threshold <- attendance$Unk / (attendance$pupils *  14) >= 0.01
attendance[attendance_data_threshold, "prop_absent_ill"] <- NA

attendance$arm <- "None"
attendance[attendance$School_AnonID %in% control_school_codes, "arm"] <- "Control" 
attendance[attendance$School_AnonID %in% hepa_school_codes, "arm"] <- "HEPA"

msoa_rates_link <- "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E08000032&metric=newCasesBySpecimenDateRollingRate&format=csv"
keep_cols <- c("msoa", "WeekStart", "covid_msoa_rate")
msoa_rates <- read_csv(msoa_rates_link) %>%
    mutate(WeekStart = date - 5) %>%
    rename(covid_msoa_rate = newCasesBySpecimenDateRollingRate, msoa = areaCode) %>%
    select(all_of(keep_cols))

cov_age_rate_link <- "https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=E08000032&metric=newCasesBySpecimenDateAgeDemographics&format=csv&release=2022-04-29"
cov_age_rates <- read_csv(cov_age_rate_link) %>%
    filter(age %in% c("05_09", "10_14")) %>%
    filter(weekdays(date) == "Saturday") %>%
    mutate(WeekStart = date - 5) %>%
    group_by(WeekStart) %>%
    summarise(covid_age_rate = mean(rollingRate))

attendance <- attendance %>% 
    left_join(msoa_rates, by=c("WeekStart", "msoa")) %>%
    left_join(cov_age_rates, by=c("WeekStart")) 

co2_sql <- "SELECT School_ID, week_start, co2_mean FROM `yhcr-prd-phm-bia-core.CY_CLASS_ACT.stats_week_school`"
co2_table <- bq_project_query(project_id, co2_sql)
co2 <- bq_table_download(co2_table)

co2 <- co2 %>% 
    rename(WeekStart = week_start, School_AnonID = School_ID) 

attendance <- attendance %>% 
    left_join(co2, by=c("WeekStart", "School_AnonID"))

thresholds <- attendance %>% group_by(WeekStart) %>% 
    summarise(outlier_threshold = quantile(prop_absent_ill, p=0.75, na.rm=TRUE) + 3*(IQR(prop_absent_ill, na.rm=TRUE)))

attendance <- attendance %>% 
    left_join(thresholds, by="WeekStart") %>% 
    mutate(covid_age_rate_scaled = covid_age_rate / max(covid_age_rate), 
           covid_msoa_rate_scaled = covid_msoa_rate / max(covid_msoa_rate, na.rm=TRUE), 
           prop_absent_ill_scaled = prop_absent_ill / max(prop_absent_ill, na.rm=TRUE), 
           co2_mean_scaled = co2_mean / max(co2_mean, na.rm=TRUE), 
           is_outlier = prop_absent_ill > outlier_threshold)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
ggplot(attendance) + 
    geom_boxplot(aes(x=WeekStart, 
                     y=covid_msoa_rate, 
                     group=WeekStart), 
                 color="blue") +
    geom_line(aes(x=WeekStart, y=covid_age_rate), color="red")


In [ ]:
ggplot(attendance, aes(x=log(covid_msoa_rate), 
                       y=log(prop_absent_ill), 
                       color=arm)) + 
    geom_point(alpha=0.4) +
    geom_smooth(method="lm")

In [ ]:
ggplot(attendance, aes(x=log(covid_msoa_rate), 
                       y=log(prop_absent_ill))) + 
    geom_point(alpha=0.4) +
    geom_smooth(method="lm")

In [ ]:
ggplot(attendance, aes(x=log(covid_age_rate), 
                       y=log(prop_absent_ill), 
                       color=arm)) + 
    geom_jitter(alpha=0.4, width=0.03) +
    geom_smooth(method="lm")

In [ ]:
ggplot(attendance, aes(x=co2_mean, 
                       y=log(prop_absent_ill), 
                       color=arm)) + 
    geom_jitter(alpha=0.4, width=0.03) +
    geom_smooth(method="lm")

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
plot_1 <- (ggplot(
    data=attendance %>% 
        mutate(month_year = as.Date(format(attendance$WeekStart, "%Y-%m-01")))
    ) + geom_boxplot(mapping=aes(x=month_year,  
                                 y=prop_absent_ill,  
                                 color=arm,  
                                 group=interaction(month_year, arm)), 
                   )
    + labs(x="", y="Illness Ratio")
     + ggtitle("Distribution of Illness Ratios by Month and Study Arm") 
     + theme(plot.title = element_text(hjust = 0.5))
 )

plot_2 <- (ggplot(attendance)
           + geom_smooth(mapping=aes(x=WeekStart, y=prop_absent_ill, color=arm), method="loess") 
           + labs(x="", y="Illness Ratio") 
           + ggtitle("LOESS Curves of Illness Ratio by Date: UVC vs Control")  
           + theme(plot.title = element_text(hjust = 0.5))
)

grid.arrange(plot_2, plot_1, nrow=2, heights=c(1.5,2))

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
ggplot(attendance) + 
    geom_boxplot(aes(x=WeekStart, y=prop_absent_ill, group=WeekStart, color="Illness Ratio"), color="black") +
    geom_line(data=attendance %>% filter(School_AnonID == "H07"), 
               mapping=aes(x=WeekStart, y=prop_absent_ill, group="School H07"), color="red") +
    geom_point(data=attendance %>% filter(School_AnonID == "H07"), 
               mapping=aes(x=WeekStart, y=prop_absent_ill), color="red")

In [ ]:
return_illness_model <- function(data, covariates) {
    model_string <- append("prop_absent_ill ~ 1 + arm", covariates)
    model_string <- append(model_string, "(1 | School_AnonID)")
    model_string <- paste(model_string, collapse=" + ")
    model <- brm(model_string, data = data, family=Gamma(link="log"),  
                 warmup = 500, iter = 2000, chains = 5, init = 1, cores = 5)
    model <- add_criterion(model, "loo")
    return(model)
}

msoa_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_msoa_rate_scaled"
)
int_msoa_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_msoa_rate_scaled*arm"
)
co2_model <- return_illness_model(
    attendance %>% na.omit(),  
    "co2_mean_scaled"
)
int_co2_model <- return_illness_model(
    attendance %>% na.omit(),  
    "co2_mean_scaled*arm"
)
msoa_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled")
)
int_msoa_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled")
)
msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled*arm")
)
int_msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled*arm")
)
age_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_age_rate_scaled"
)
int_age_model <- return_illness_model(
    attendance %>% na.omit(), 
    "covid_age_rate_scaled*arm"
)
age_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled", "co2_mean_scaled")
)
int_age_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled")
)
age_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled", "co2_mean_scaled*arm")
)
int_age_int_co2_model <- return_illness_model(
    attendance %>% na.omit(), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled*arm")
)

In [ ]:
loo_compare(msoa_model,  
            int_msoa_model,  
            co2_model,  
            int_co2_model,  
            msoa_co2_model,  
            int_msoa_co2_model,  
            msoa_int_co2_model,  
            int_msoa_int_co2_model,  
            age_model,  
            int_age_model,  
            age_co2_model,  
            int_age_co2_model,  
            age_int_co2_model,  
            int_age_int_co2_model,
            criterion="loo")

In [ ]:
msoa_int_co2_model

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
color_scheme_set("darkgray")
plot <- mcmc_areas(msoa_int_co2_model, 
                   regex_pars="^b_*", 
                   prob=.95)
plot 

In [ ]:
names(msoa_int_co2_model$fit)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)

plot_posterior_densities <- function(model_tab) {
    ggplot(model_tab %>%  
               filter(!grepl("\\[", Parameter)) %>% 
               filter(grepl("b_", Parameter))) + 
    geom_density(aes(x=value, fill=Parameter),  
                 alpha=0.5) +  
    geom_vline(xintercept=0, linetype="dashed", col="red") + 
    facet_grid(Parameter ~.) + 
    theme_light() 
}

plot_posterior_densities <- function(model_tab) {
    model_tab %>%   
        filter(!grepl("\\[", Parameter)) %>%  
        filter(grepl("b_", Parameter)) %>%
    ggplot(aes(x=value, y=Parameter, fill=Parameter)) +
    stat_halfeye(show.legend=FALSE) +
    geom_vline(xintercept=0, linetype="dashed", col="red") + 
    theme_light() 
}

plot_caterpillars <- function(model_tab) {
    ggplot(model_tab %>% 
               filter(Parameter %in% c("b_armHEPA", "b_covid_rate", "b_Intercept", "b_co2_mean")),
           aes(x=Iteration, y=value, col=as.factor(Chain))) +
      geom_line() +
      facet_grid(Parameter ~ .) +
      labs(title = "Caterpillar Plots", 
           col   = "Chains")
}

best_model_tab <- ggs(msoa_int_co2_model)
plot_posterior_densities(best_model_tab)

In [ ]:
?stat_slab

In [ ]:
ggplot(best_model_tab %>%  
           filter(Parameter == "b_armHEPA")) +
geom_density(aes(x=value, fill=Parameter), 
                 alpha=0.5) +  
geom_vline(xintercept=0, linetype="dashed") + 
theme_light() 

In [ ]:
posterior_interval(best_model_tab %>% filter(Parameter == "b_armHEPA"))

In [ ]:
spread(best_model_tab)

In [ ]:
as_matrix
(best_model_tab)

In [ ]:
plot_caterpillars(best_model_tab)

In [ ]:
no_msoa_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    "covid_msoa_rate_scaled"
)
no_int_msoa_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    "covid_msoa_rate_scaled*arm"
)
no_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier),  
    "co2_mean_scaled"
)
no_int_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier),  
    "co2_mean_scaled*arm"
)
no_msoa_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled")
)
no_int_msoa_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled")
)
no_msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_msoa_rate_scaled", "co2_mean_scaled*arm")
)
no_int_msoa_int_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_msoa_rate_scaled*arm", "co2_mean_scaled*arm")
)
no_age_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    "covid_age_rate_scaled"
)
no_int_age_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    "covid_age_rate_scaled*arm"
)
no_age_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_age_rate_scaled", "co2_mean_scaled")
)
no_int_age_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled")
)
no_age_int_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_age_rate_scaled", "co2_mean_scaled*arm")
)
no_int_age_int_co2_model <- return_illness_model(
    attendance %>% na.omit() %>% filter(!is_outlier), 
    c("covid_age_rate_scaled*arm", "co2_mean_scaled*arm")
)

In [ ]:
loo_compare(no_msoa_model,  
            no_int_msoa_model,  
            no_co2_model,  
            no_int_co2_model,  
            no_msoa_co2_model,  
            no_int_msoa_co2_model,  
            no_msoa_int_co2_model,  
            no_int_msoa_int_co2_model,  
            no_age_model,  
            no_int_age_model,  
            no_age_co2_model,  
            no_int_age_co2_model,  
            no_age_int_co2_model,  
            no_int_age_int_co2_model,
            criterion="loo")

In [ ]:
best_no_out_model_tab <- ggs(no_msoa_model)
plot_posterior_densities(best_no_out_model_tab)

In [ ]:
plot_caterpillars(best_no_out_model_tab)

In [ ]:
prior_summary(basic_model_msoa_rate)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(brm_model) %>%
  ggplot(aes(x = .row, y = .residual)) +
  stat_pointinterval()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
attendance %>%
  add_residual_draws(brm_model) %>%
  median_qi() %>%
  ggplot(aes(sample = .residual)) +
  geom_qq() +
  geom_qq_line()

In [ ]:
glmer_model <- glmer(formula = prop_absent_ill ~ 1 + arm + covid_rate + (1 | School_AnonID),    
                     data = attendance, 
                     family=Gamma(link="log"))

In [ ]:
summary(glmer_model)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
plot(fitted(glmer_model), resid(glmer_model, type = "pearson"))# this will create the plot
abline(0,0, col="red")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
qqnorm(resid(glmer_model)) 
qqline(resid(glmer_model), col = "red") # add a perfect fit line

In [ ]:
msoa_rates_link <- "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E08000032&metric=newCasesBySpecimenDateRollingRate&format=csv"
msoa_rates <- read_csv(msoa_rates_link)


In [ ]:
head(msoa_rates)